In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
from torch.utils.data import DataLoader, random_split

# Dataset disponivel em https://www.kaggle.com/code/gauravduttakiit/resnet50-corals-image-classification
train_dir = '/home/guilherme/Documentos/Projetos/Estudos-IC/Deep Learning for CV/Praticing'

# Definindo as transformações para as imagens
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Carregando o dataset com todas as imagens
full_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)

# Determinando o tamanho de treino e validação (80% treino, 20% validação)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

# Dividindo o dataset em treino e validação
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Aplicando as transformações de validação no dataset de validação
val_dataset = datasets.ImageFolder(root=train_dir, transform=val_transform)

# Criando DataLoader para carregar as imagens em lotes
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Verificando o tamanho dos datasets
print(f'Número de imagens de treinamento: {len(train_dataset)}')
print(f'Número de imagens de validação: {len(val_dataset)}')

# Exemplo de iteração sobre os dados de treinamento
for images, labels in train_loader:
    print(images.shape)  # Exibe o formato das imagens (batch_size, 3, 128, 128)
    print(labels)         # Exibe os rótulos das imagens
    break


Número de imagens de treinamento: 738
Número de imagens de validação: 923
torch.Size([32, 3, 128, 128])
tensor([0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 1, 1, 0])


Criando o modelo

In [4]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2, stride=2),
    torch.nn.Flatten(),
    torch.nn.Linear(16 * 64 * 64, 2)
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader

criterion = nn.CrossEntropyLoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train(model, train_loader, criterion, optimizer, device):
    model.train()  
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Média da perda e acurácia
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct / total * 100
    
    return epoch_loss, epoch_accuracy

In [5]:
for epoch in range(50):
    epoch_loss, epoch_accuracy = train(model, train_loader, criterion, optimizer, device)
    
    print(f"Época {epoch+1}/500 - Perda: {epoch_loss:.4f} - Acurácia: {epoch_accuracy:.2f}%")

Época 1/500 - Perda: 1.2012 - Acurácia: 68.29%
Época 2/500 - Perda: 0.8470 - Acurácia: 71.82%
Época 3/500 - Perda: 0.7750 - Acurácia: 72.22%
Época 4/500 - Perda: 1.2125 - Acurácia: 67.34%
Época 5/500 - Perda: 0.9405 - Acurácia: 70.60%
Época 6/500 - Perda: 0.6429 - Acurácia: 73.98%
Época 7/500 - Perda: 0.6635 - Acurácia: 72.36%
Época 8/500 - Perda: 0.8823 - Acurácia: 70.73%
Época 9/500 - Perda: 0.8938 - Acurácia: 67.75%
Época 10/500 - Perda: 0.4861 - Acurácia: 77.64%
Época 11/500 - Perda: 0.4778 - Acurácia: 79.95%
Época 12/500 - Perda: 0.4203 - Acurácia: 81.98%
Época 13/500 - Perda: 0.4207 - Acurácia: 81.44%
Época 14/500 - Perda: 0.4184 - Acurácia: 81.03%
Época 15/500 - Perda: 0.5332 - Acurácia: 77.10%
Época 16/500 - Perda: 0.4680 - Acurácia: 80.49%
Época 17/500 - Perda: 0.4365 - Acurácia: 80.89%
Época 18/500 - Perda: 0.4801 - Acurácia: 82.11%
Época 19/500 - Perda: 0.4827 - Acurácia: 77.51%
Época 20/500 - Perda: 0.4047 - Acurácia: 81.71%
Época 21/500 - Perda: 0.3559 - Acurácia: 85.09%
É

In [6]:
import torch
from torch.utils.data import DataLoader

# Função de avaliação
def evaluate(model, val_loader, criterion, device):
    model.eval()  # Coloca o modelo em modo de avaliação (sem gradientes)
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # Não calcular os gradientes durante a validação
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    # Média da perda e acurácia
    val_loss = running_loss / len(val_loader)
    val_accuracy = correct / total * 100
    
    return val_loss, val_accuracy

# Testando com os dados de validação após o treinamento
val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)

print(f'Loss de validação: {val_loss:.4f}')
print(f'Acurácia de validação: {val_accuracy:.2f}%')


Loss de validação: 0.2749
Acurácia de validação: 90.03%


In [8]:
from torchviz import make_dot
import torch

# Definindo uma entrada fictícia para passar pelo modelo (a entrada deve ter o formato correto)
x = torch.randn(1, 3, 128, 128).to(device)  # 1 imagem com 3 canais (RGB) e 128x128 pixels

# Passando a entrada pela rede para gerar o gráfico
y = model(x)

# Gerando o gráfico visual
make_dot(y, params=dict(model.named_parameters())).render("cnn_model", format="png")

# Isso irá gerar o arquivo 'cnn_model.png' na pasta do seu projeto


'cnn_model.png'